In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings('ignore')

plt.style.use('ggplot')

In [2]:
cex_list = ['Bitmax', 'Binance', 'BingX', 'Bitbox', 
 'Bitfinex', 'BitGo', 'Bitkub', 'BitPanda',
 'Bitrue', 'Bitso', 'Bitstamp', 'Bittrex',
 'Bitvavo', 'BitZ', 'BKEX', 'Blockchain.com', 
 'BTC Markets', 'BTCTurk', 'ByBit', 'Changelly',
 'Coinbase', 'Coinex', 'Coinlist', 'CoinSpot',
 'Crypto.com', 'Gate.io', 'Hotbit', 'Indodax',
 'Koibanx', 'Korbit', 'Kraken', 'Kucoin',
 'MEXC', 'Netcoins', 'OkCoin', 'OKX',
 'Paribu', 'Phemex', 'ProBit', 'Ripio', 
 'UpHold', 'Vindax', 'WhiteBIT']



In [13]:
# Load the data into a DataFrame
df = pd.read_csv('CEX_balances/result.csv')

# Convert 'dt' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Pivot the DataFrame to have 'entity_name' as columns and 'dt' as the index
pivot_df = df.pivot(index='Date', columns='Entity', values='AlgoBalance')

# Create a new column for total balance by summing across entities
pivot_df['Total'] = pivot_df.sum(axis=1)
pivot_df.reset_index(inplace=True)

In [16]:
pivot_df.to_csv('data/CEX_balances.csv', index=False)

In [15]:
pivot_df['Date'] = pd.to_datetime(pivot_df['Date'])
last_date = pivot_df['Date'].max()

# Step 3: Get the last day balances
last_day_df = pivot_df[pivot_df['Date'] == last_date]

# Step 4: Prepare ranking DataFrame
ranking_data = {'CEX': [], 'Balance': [], '24h%': [], '7d%': [], '30d%': [], '90d%': []}

# Get the list of CEX columns (excluding 'Date')
cex_columns = [col for col in pivot_df.columns if col != 'Date']

# Step 5: Populate the ranking data
for cex in cex_columns:
    # Extract the last day balance
    balance = last_day_df[cex].values[0]
    
    # Add CEX name and Balance
    ranking_data['CEX'].append(cex.replace(' Balance', ''))
    ranking_data['Balance'].append(balance)
    
    # Calculate 24-hour change
    previous_day = last_date - pd.Timedelta(days=1)
    previous_day_balance = pivot_df[pivot_df['Date'] == previous_day][cex]
    
    if not previous_day_balance.empty and previous_day_balance.values[0] != 0:
        change_24h = ((balance - previous_day_balance.values[0]) / previous_day_balance.values[0]) * 100
    else:
        change_24h = None
    ranking_data['24h%'].append(change_24h)
    
    # Calculate percentage changes for 7d, 30d, and 90d
    for days in [7, 30, 90]:
        past_date = last_date - pd.Timedelta(days=days)
        past_balance = pivot_df[pivot_df['Date'] == past_date][cex]
        
        if not past_balance.empty and past_balance.values[0] != 0:
            change = ((balance - past_balance.values[0]) / past_balance.values[0]) * 100
        else:
            change = None
        
        ranking_data[f'{days}d%'].append(change)

# Step 6: Create the ranking DataFrame
ranking_df = pd.DataFrame(ranking_data)

# Step 7: Sort the ranking DataFrame by Balance in descending order
ranking_df.sort_values(by='Balance', ascending=False, inplace=True)
ranking_df.reset_index(drop=True, inplace=True)
ranking_df.to_csv('data/CEX_ranking.csv', index=False)